In [1]:
import h5py

In [2]:
import numpy as np

In [3]:
from pyiron_atomistics import Project

In [4]:
pr = Project("test")

In [5]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [7]:
structure.set_repeat([9,9,9])

In [8]:
len(structure)

2916

In [9]:
job = pr.create.job.Lammps("lmp")

In [10]:
job.structure =structure

In [11]:
job.list_potentials()

['1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1',
 '1996--Farkas-D--Nb-Ti-Al--LAMMPS--ipr1',
 '1997--Liu-X-Y--Al-Mg--LAMMPS--ipr1',
 '1998--Liu-X-Y--Al-Mg--LAMMPS--ipr1',
 '1999--Liu-X-Y--Al-Cu--LAMMPS--ipr1',
 '1999--Mishin-Y--Al--LAMMPS--ipr1',
 '2000--Landa-A--Al-Pb--LAMMPS--ipr1',
 '2000--Sturgeon-J-B--Al--LAMMPS--ipr1',
 '2002--Mishin-Y--Ni-Al--LAMMPS--ipr1',
 '2003--Lee-B-J--Al--LAMMPS--ipr1',
 '2003--Zope-R-R--Al--LAMMPS--ipr1',
 '2003--Zope-R-R--Ti-Al--LAMMPS--ipr1',
 '2004--Liu-X-Y--Al--LAMMPS--ipr1',
 '2004--Mishin-Y--Ni-Al--LAMMPS--ipr1',
 '2004--Mishin-Y--Ni-Al--LAMMPS--ipr2',
 '2004--Zhou-X-W--Al--LAMMPS--ipr2',
 '2005--Mendelev-M-I--Al-Fe--LAMMPS--ipr1',
 '2007--Silva-A-C--Al-Ni--LAMMPS--ipr1',
 '2008--Mendelev-M-I--Al--LAMMPS--ipr1',
 '2009--Kim-Y-M--Mg-Al--LAMMPS--ipr1',
 '2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1',
 '2009--Purja-Pun-G-P--Ni-Al--LAMMPS--ipr1',
 '2009--Zhakhovskii-V-V--Al--LAMMPS--ipr2',
 '2010--Lee-E--Fe-Al--LAMMPS--ipr1',
 '2010--Mendelev-M-I--fictional-Al-1--

In [12]:
job.potential = '2002--Mishin-Y--Ni-Al--LAMMPS--ipr1'

In [13]:
job.calc_md(n_ionic_steps=1000, n_print=10, temperature=500.0)

In [14]:
job.run()

The job lmp was saved and received the ID: lmp


In [15]:
file_name = job.project_hdf5.file_name
file_name

'/Users/janssen/pyiron/projects/2023/2023-05-02-pyiron-start/test/lmp.h5'

In [16]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

In [17]:
with h5py.File(file_name, 'r') as f:
    f.visititems(print_attrs)

lmp
lmp/HDF_VERSION
    TITLE: unicode
lmp/NAME
    TITLE: unicode
lmp/TYPE
    TITLE: unicode
lmp/VERSION
    TITLE: unicode
lmp/executable
lmp/executable/HDF_VERSION
    TITLE: unicode
lmp/executable/NAME
    TITLE: unicode
lmp/executable/OBJECT
    TITLE: unicode
lmp/executable/TYPE
    TITLE: unicode
lmp/executable/executable
lmp/executable/executable/HDF_VERSION
    TITLE: unicode
lmp/executable/executable/NAME
    TITLE: unicode
lmp/executable/executable/OBJECT
    TITLE: unicode
lmp/executable/executable/READ_ONLY
    TITLE: int
lmp/executable/executable/TYPE
    TITLE: unicode
lmp/executable/executable/VERSION
    TITLE: unicode
lmp/executable/executable/accepted_return_codes__index_5
    TITLE: json
lmp/executable/executable/executable__index_3
    TITLE: None
lmp/executable/executable/mpi__index_4
    TITLE: int
lmp/executable/executable/name__index_1
    TITLE: unicode
lmp/executable/executable/operation_system_nt__index_2
    TITLE: int
lmp/executable/executable/version__in

In [18]:
with h5py.File(file_name, 'r') as f:
    cells = f["lmp/output/generic/cells"][:]
    positions = f["lmp/output/generic/positions"][:]
    energy_tot = f["lmp/output/generic/energy_tot"][:]
    energy_pot = f["lmp/output/generic/energy_pot"][:]
    forces = f["lmp/output/generic/forces"][:]
    

In [19]:
np.all(cells == job.output.cells)

True

In [20]:
np.all(positions == job.output.positions)

True

In [21]:
np.all(energy_tot == job.output.energy_tot)

True

In [22]:
np.all(energy_pot == job.output.energy_pot)

True

In [23]:
np.all(forces == job.output.forces)

True